# Load triples to HANA Cloud KG Engine

In [32]:
#Credentials
address = '291933b3-a304-499f-8649-14ec7ce3ce9a.hana.prod-ap10.hanacloud.ondemand.com'
port = '443'
user = 'DBUSER'
password = 'Testing123'

In [33]:
from hdbcli import dbapi

# connect to database using username/password
conn = dbapi.connect(user=user, password=password, address=address, port=port)
cursor = conn.cursor()

In [34]:
#Populate a new RDF in HANA Cloud with the ttl file content. We put the ontology into a specific graph.
ttl_filename = "/Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/ontology/ontology-nsmen.ttl"
graph_name = "http://www.semanticweb.org/ontologies/2025/advisory-rdf-test"

In [35]:
# Drop existing one if needed
query = """drop graph <{graph_name}>""". format(graph_name=graph_name)
print(query)

drop graph <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test>


In [36]:
#Uncomment to drop the ontology
#resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, '', '?', None) )

In [37]:
with open(ttl_filename, 'r') as ttlfp:
    request_hdrs = ''
    request_hdrs += 'rqx-load-protocol: true' + '\r\n'            # required header for upload protocol
    request_hdrs += 'rqx-load-filename: ' + ttl_filename + '\r\n' # optional header
    request_hdrs += 'rqx-load-graphname: ' + graph_name + '\r\n'   # optional header to specify name of the graph, 
                                                                  #if not provided RDF data will be loaded to internal-default-graph
    conn.cursor().callproc('SPARQL_EXECUTE', (ttlfp.read(), request_hdrs, '?', None))

In [38]:
query = """
    SELECT * 
    FROM SPARQL_TABLE('
        PREFIX : <http://www.semanticweb.org/ontologies/2025/advisory-ontology-test/>
        SELECT ?s 
        FROM <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test> 
        WHERE { 
            ?s a :NSman
            }
        '
        )"""

In [39]:
#There is just one individual
cursor.execute(query)
for row in cursor:
    print(row)

In [40]:
#Function to load more ttl files
def load_file(ttl_filename, graph_name):
    try:
        with open(ttl_filename, 'r') as ttlfp:
            request_hdrs = ''
            request_hdrs += 'rqx-load-protocol: true' + '\r\n'            # required header for upload protocol
            request_hdrs += 'rqx-load-filename: ' + ttl_filename + '\r\n' # optional header
            request_hdrs += 'rqx-load-graphname: ' + graph_name + '\r\n'  # optional header to specify name of the graph, 
                                                                          # if not provided RDF data will be loaded 
                                                                          # to internal-default-graph
            conn.cursor().callproc('SPARQL_EXECUTE', (ttlfp.read(), request_hdrs, '?', None))
        return 0
    except:
        print("An exception occurred")
        return 1

In [41]:
query = """drop graph <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test>"""

In [14]:
#Uncomment to drop the rdf
#resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, '', '?', None) )

In [42]:
import glob

path = "/Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/"

list = sorted(glob.glob(path + "*.ttl"))
print(list[5])
print(len(list))

/Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/unit_instances.ttl
6


In [43]:
for i in range(len(list)):
    print(i, list[i])
    ttl_filename = list[i]
    graph_name = "http://www.semanticweb.org/ontologies/2025/advisory-rdf-test"
    load_file(ttl_filename, graph_name)

0 /Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/booking_instances.ttl
An exception occurred
1 /Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/nsman_instances.ttl
An exception occurred
2 /Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/pesstatus_instances.ttl
An exception occurred
3 /Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/policy_instances.ttl
An exception occurred
4 /Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instance-nsmen/rank_

In [44]:
# Complex query using your existing prefix and FROM
query = """
PREFIX : <http://www.semanticweb.org/ontologies/2025/advisory-ontology-test/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX schema: <https://schema.org/>

SELECT ?nsman ?name ?unit ?rank ?pesStatus ?booking ?bookingTimeStart ?bookingTimeEnd
FROM <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test>
WHERE {
    ?nsman a :NSMan .
    
    # Get basic info if it exists
    OPTIONAL { ?nsman foaf:name ?name . }
    OPTIONAL { ?nsman :servesIn ?unit . }
    OPTIONAL { ?nsman :hasRank ?rank . }
    OPTIONAL { ?nsman :hasPESStatus ?pesStatus . }
    
    # Get booking information if available
    OPTIONAL {
        ?nsman :hasBooking ?booking .
        ?booking :hasTime ?bookingTime .
        ?bookingTime time:hasBeginning ?bookingTimeStart .
        ?bookingTime time:hasEnd ?bookingTimeEnd .
    }
}
LIMIT 20
"""


In [45]:
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

nsman,name,unit,rank,pesStatus,booking,bookingTimeStart,bookingTimeEnd



In [46]:
# Count the number of triples in the rdf
query = """
    SELECT (COUNT(*) as ?Triples) 
    WHERE 
      { GRAPH <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test> 
          { ?s ?p ?o } 
      }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

Triples
170



In [47]:
# Obtain a list of classes in the ontology
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test/> 
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    
    SELECT DISTINCT ?c
    FROM <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test>
    WHERE {
        ?c a owl:Class
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

c
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ServiceType
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/PolicyRule
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ExitPermit
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ObligationRule
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/SGT
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/Location
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/Unit
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/Deferment
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ExemptionRule
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/PES_F
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/PES_C
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/PES_E
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/CPL
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/EligibilityRule
http://www.semanticweb.org/ontologies/2025/nsmen-ontology

In [48]:
# Query all object properties defined in your NSMen ontology
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX : <http://www.semanticweb.org/ontologies/2025/nsmen-ontology/>

SELECT DISTINCT ?p
WHERE {
    ?p a owl:ObjectProperty .
    FILTER(STRSTARTS(STR(?p), "http://www.semanticweb.org/ontologies/2025/nsmen-ontology/"))
}
"""

# Execute SPARQL
resp = conn.cursor().callproc(
    'SPARQL_EXECUTE',
    (query, 'Accept: application/sparql-results+csv', '?', None)
)

# Print results (excluding only the header row)
lines = resp[2].splitlines()
if len(lines) > 1:
    for line in lines[1:]:
        print(line)
else:
    print("No object properties found. Make sure your TTL is loaded into the RDF graph.")


http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasRank
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasPESStatus
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/occursAt
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/appliesTo
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasBooking
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasObligation
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasScope
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasExitPermit
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/servesIn
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasTime
http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasDeferment


## Other queries

In [49]:
# Get the onthology schema 
query = """
    CONSTRUCT {?s ?p ?o} FROM <http://www.semanticweb.org/ontologies/2025/advisory-rdf-test> 
    WHERE {?s ?p ?o}
    LIMIT 10
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2])

@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ServiceType> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/PolicyRule> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ExitPermit> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/ObligationRule> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/SGT> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/nsmen-ontology/hasExitPermit> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002

In [ ]:
import pandas as pd
from hana_ml.dataframe import ConnectionContext, create_dataframe_from_pandas

# 1. Load CSV
df = pd.read_csv("/Users/i757416/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/backend/db/content_advisories_table.sql.csv")  # your CSV file

conn = ConnectionContext(
    address="291933b3-a304-499f-8649-14ec7ce3ce9a.hana.prod-ap10.hanacloud.ondemand.com",
    port=443,
    user="DBUSER",
    password="Testing123",
)

# 3. Upload CSV into HANA table
df_hana = create_dataframe_from_pandas(
    connection_context=conn,   # <-- correct parameter name
    pandas_df=df,
    table_name="ADVISORIES4",
    schema="DBUSER",
    force=True  # overwrite if table exists
)

print("Upload successful!")

100%|██████████| 1/1 [00:00<00:00,  3.58it/s]


AttributeError: 'int' object has no attribute 'collect'